In [2]:
import sys
sys.path.insert(0, '../' )

In [3]:
from libgf264fft import clib_wrapper as cl

In [4]:
cl.gf264_mul( cl.index_to_gf264(1) , cl.index_to_gf264(1) )

1

In [5]:
import random

def _rand_bytes(l) :
  return bytes( [random.randrange(256) for i in range(l)] )


In [13]:
from aes128 import aes128_encrypt

#pt = _rand_bytes(16)
#sk = _rand_bytes(16)
pt = bytes( [i*16+i for i in range(16)] )
sk = bytes( [i for i in range(16)] )

ct, rd_states , rk , rk_tmp = aes128_encrypt( pt , sk )

print( sk )
print( pt )
print( rd_states , rk , rk_tmp  )
print( "ct0", ct )


from Crypto.Cipher import AES
ct2 = AES.new( sk , AES.MODE_ECB ).encrypt( pt )
print( "ct2" , ct2 )


b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
b'\x00\x11"3DUfw\x88\x99\xaa\xbb\xcc\xdd\xee\xff'
b'_rd\x15W\xf5\xbc\x92\xf7\xbe;)\x1d\xb9\xf9\x1a\xff\x87\x96\x841\xd8jQdQQ\xfaw:\xd0\tL\x9c\x1ef\xf7q\xf0v,?\x86\x8eSM\xf2Vc\x85\xb7\x9f\xfcS\x8d\xf9\x97\xbeG\x8euG\xd6\x91\xf4\xbc\xd4T2\xe5T\xd0u\xf1\xd6\xc5\x1d\xd0;<\x98\x16\xeet\x00\xf8\x7fUk,\x04\x9c\x8eZ\xd06\xc5~\x1c\x15\x9a\x9b\xd2\x86\xf0_K\xe0\x98\xc649\xba\xa0=\xe7\xa1\xf9\xb5n\xd5Q,\xba_AM#\xe9\xf7N\xec\x020 \xf6\x1b\xf2\xcc\xf25<!\xc7' b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\xd6\xaat\xfd\xd2\xafr\xfa\xda\xa6x\xf1\xd6\xabv\xfe\xb6\x92\xcf\x0bd=\xbd\xf1\xbe\x9b\xc5\x00h0\xb3\xfe\xb6\xfftN\xd2\xc2\xc9\xbflY\x0c\xbf\x04i\xbfAG\xf7\xf7\xbc\x955>\x03\xf9l2\xbc\xfd\x05\x8d\xfd<\xaa\xa3\xe8\xa9\x9f\x9d\xebP\xf3\xafW\xad\xf6"\xaa^9\x0f}\xf7\xa6\x92\x96\xa7U=\xc1\n\xa3\x1fk\x14\xf9p\x1a\xe3_\xe2\x8cD\n\xdfMN\xa9\xc0&GC\x875\xa4\x1ce\xb9\xe0\x16\xba\xf4\xae\xbfz\xd2T\x992\xd1\xf0\x85Wh\x10\x93\xed\x9c\xb

In [8]:
from r1cs import *

################  test   #####################################


r1cs = get_r1cs(4096, 4096, 64, 128)

In [10]:
mat_a = r1cs['A']
mat_b = r1cs['B']
mat_c = r1cs['C']

def is_vec_zero( vec ):
    r = 0
    for e in vec : r |= e
    return 0 == r

def is_colvec_zero( mat , col_idx ):
    r = 0
    for i in range(len(mat)):
        r |= mat[i][col_idx]
    return 0 == r



mat = mat_a
print( 'mat_a' , len( mat ) , len(mat[0]) )
zero_cols = 0
for i in range(64):
    if is_colvec_zero( mat , i ) :
        print( f'{i},' , end = '' )
        zero_cols += 1
print()
print('zero_cols:', zero_cols)

mat = mat_b
print( 'mat_b' , len( mat ) , len(mat[0]) )
zero_cols = 0
for i in range(64):
    if is_colvec_zero( mat , i ) :
        print( f'{i},' , end = '' )
        zero_cols += 1
print()
print('zero_cols:', zero_cols)

mat = mat_c
print( 'mat_c' , len( mat ) , len(mat[0]) )
zero_cols = 0
for i in range(64):
    if is_colvec_zero( mat , i ) :
        print( f'{i},' , end = '' )
        zero_cols += 1
print()
print('zero_cols[/64]:', zero_cols)

mat_a 4096 4096
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
zero_cols: 47
mat_b 4096 4096
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
zero_cols: 63
mat_c 4096 4096
17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
zero_cols: 47


In [25]:

import aes128R1CS_z

import importlib
importlib.reload( aes128R1CS_z )

z = aes128R1CS_z.get_vec_z(pt,sk,64)


In [26]:
def mat_x_vec( mat , vec , n_col=-1) :
    if 0 > n_col : n_col = len(mat)
    r = [0]*n_col
    for i in range(n_col):
        ri = 0
        for j in range(len(vec)):
            ri ^= cl.gf264_mul( mat[i][j] , vec[j] )
        r[i] = ri
    return r

#########  test  ############

Az = mat_x_vec( mat_a , z )
Bz = mat_x_vec( mat_b , z )
Cz = mat_x_vec( mat_c , z )

In [27]:
for i in range(4096):
    ci = cl.gf264_mul( Az[i] , Bz[i] ) ^ Cz[i]
    if ci :
        print( f"{i}: fails.  Az[{i}]:{hex(Az[i])} x Bz[{i}]:{hex(Bz[i])} ^ Cz[{i}]:{hex(Cz[i])} = {hex(ci)}" )
        break
